<a href="https://colab.research.google.com/github/denistoo749/Used-Car-Price-Prediction/blob/main/used_car_price_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Problem Definition
predict the price of used cars based on various attributes.

# Data

https://www.kaggle.com/competitions/playground-series-s4e9/data

# Evaluation
Root Mean Squared Error (RMSE)
Submissions are scored on the root mean squared error. RMSE is defined as:

$$
\text{RMSE} = \sqrt{\frac{1}{N} \sum_{i=1}^{N} (y_i - \hat{y}_i)^2}
$$

$$
\text{RMSE} = \left( \frac{1}{N} \sum_{i=1}^{N} (y_i - \hat{y}_i)^2 \right)^{\frac{1}{2}}
$$

$$
\text{Where }  \hat{y}_i \text{ is the predicted value and } {y}_i \text{ is the original value for each instance i. }
$$

In [1]:
# Unzip file
!unzip '/content/drive/MyDrive/Used Car Price Prediction/playground-series-s4e9.zip' -d '/content/drive/MyDrive/Used Car Price Prediction/data/'

Archive:  /content/drive/MyDrive/Used Car Price Prediction/playground-series-s4e9.zip
  inflating: /content/drive/MyDrive/Used Car Price Prediction/data/sample_submission.csv  
  inflating: /content/drive/MyDrive/Used Car Price Prediction/data/test.csv  
  inflating: /content/drive/MyDrive/Used Car Price Prediction/data/train.csv  


In [2]:
# Set Up
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [3]:
# Read the csv file
data = pd.read_csv('/content/drive/MyDrive/Used Car Price Prediction/data/train.csv')
data.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes,4200
1,1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999
2,2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,Yes,13900
3,3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000
4,4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,Yes,97500


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188533 entries, 0 to 188532
Data columns (total 13 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            188533 non-null  int64 
 1   brand         188533 non-null  object
 2   model         188533 non-null  object
 3   model_year    188533 non-null  int64 
 4   milage        188533 non-null  int64 
 5   fuel_type     183450 non-null  object
 6   engine        188533 non-null  object
 7   transmission  188533 non-null  object
 8   ext_col       188533 non-null  object
 9   int_col       188533 non-null  object
 10  accident      186081 non-null  object
 11  clean_title   167114 non-null  object
 12  price         188533 non-null  int64 
dtypes: int64(4), object(9)
memory usage: 18.7+ MB


In [5]:
data.isna().sum()

,0
id,0
brand,0
model,0
model_year,0
milage,0
fuel_type,5083
engine,0
transmission,0
ext_col,0
int_col,0


# Preprocessing

In [6]:
# Create a copy
df = data.copy()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188533 entries, 0 to 188532
Data columns (total 13 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            188533 non-null  int64 
 1   brand         188533 non-null  object
 2   model         188533 non-null  object
 3   model_year    188533 non-null  int64 
 4   milage        188533 non-null  int64 
 5   fuel_type     183450 non-null  object
 6   engine        188533 non-null  object
 7   transmission  188533 non-null  object
 8   ext_col       188533 non-null  object
 9   int_col       188533 non-null  object
 10  accident      186081 non-null  object
 11  clean_title   167114 non-null  object
 12  price         188533 non-null  int64 
dtypes: int64(4), object(9)
memory usage: 18.7+ MB


# Conver string to categories

In [7]:
# Find the columns which contains strings
for label, content in df.items():
    if pd.api.types.is_string_dtype(content):
        print(label)

brand
model
engine
transmission
ext_col
int_col


In [8]:
# This will turn all the string value into category values
for label, content in df.items():
    if pd.api.types.is_string_dtype(content):
        df[label] = content.astype('category').cat.as_ordered()

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188533 entries, 0 to 188532
Data columns (total 13 columns):
 #   Column        Non-Null Count   Dtype   
---  ------        --------------   -----   
 0   id            188533 non-null  int64   
 1   brand         188533 non-null  category
 2   model         188533 non-null  category
 3   model_year    188533 non-null  int64   
 4   milage        188533 non-null  int64   
 5   fuel_type     183450 non-null  object  
 6   engine        188533 non-null  category
 7   transmission  188533 non-null  category
 8   ext_col       188533 non-null  category
 9   int_col       188533 non-null  category
 10  accident      186081 non-null  object  
 11  clean_title   167114 non-null  object  
 12  price         188533 non-null  int64   
dtypes: category(6), int64(4), object(3)
memory usage: 12.0+ MB


In [10]:
# Convert object columns to categorical
df['fuel_type'] = df['fuel_type'].astype('category')
df['accident'] = df['accident'].astype('category')
df['clean_title'] = df['clean_title'].astype('category')

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188533 entries, 0 to 188532
Data columns (total 13 columns):
 #   Column        Non-Null Count   Dtype   
---  ------        --------------   -----   
 0   id            188533 non-null  int64   
 1   brand         188533 non-null  category
 2   model         188533 non-null  category
 3   model_year    188533 non-null  int64   
 4   milage        188533 non-null  int64   
 5   fuel_type     183450 non-null  category
 6   engine        188533 non-null  category
 7   transmission  188533 non-null  category
 8   ext_col       188533 non-null  category
 9   int_col       188533 non-null  category
 10  accident      186081 non-null  category
 11  clean_title   167114 non-null  category
 12  price         188533 non-null  int64   
dtypes: category(9), int64(4)
memory usage: 8.2 MB


# Fill Missing Values

## Filling and turning categorical variables into numbers

In [12]:
df.isna().sum()

,0
id,0
brand,0
model,0
model_year,0
milage,0
fuel_type,5083
engine,0
transmission,0
ext_col,0
int_col,0


In [13]:
# Turn categorical variables into numbers and fill missing
for label, content in df.items():
  if not pd.api.types.is_numeric_dtype(content):
    # Turn categories into numbers and add +1 (only for non-null entries)
    df[label] = pd.Categorical(content).codes + 1
    # Check if there are any missing values in the column
    if content.isna().sum() > 0:
      # Add binary column to indicate whether sample had missing values
      df[label + '_is_missing'] = pd.isnull(content)

In [14]:
df.isna().sum()

,0
id,0
brand,0
model,0
model_year,0
milage,0
fuel_type,0
engine,0
transmission,0
ext_col,0
int_col,0


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188533 entries, 0 to 188532
Data columns (total 16 columns):
 #   Column                  Non-Null Count   Dtype
---  ------                  --------------   -----
 0   id                      188533 non-null  int64
 1   brand                   188533 non-null  int8 
 2   model                   188533 non-null  int16
 3   model_year              188533 non-null  int64
 4   milage                  188533 non-null  int64
 5   fuel_type               188533 non-null  int8 
 6   engine                  188533 non-null  int16
 7   transmission            188533 non-null  int8 
 8   ext_col                 188533 non-null  int16
 9   int_col                 188533 non-null  int16
 10  accident                188533 non-null  int8 
 11  clean_title             188533 non-null  int8 
 12  price                   188533 non-null  int64
 13  fuel_type_is_missing    188533 non-null  bool 
 14  accident_is_missing     188533 non-null  bool 
 15  

In [16]:
df.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price,fuel_type_is_missing,accident_is_missing,clean_title_is_missing
0,0,32,496,2007,213000,3,117,39,313,72,2,1,4200,False,False,False
1,1,29,931,2002,143250,3,367,39,264,11,1,1,4999,False,False,False
2,2,10,1576,2002,136731,2,641,39,39,72,2,1,13900,False,False,False
3,3,17,759,2017,19500,3,864,50,30,15,2,1,45000,False,False,False
4,4,37,1078,2021,7388,3,260,24,30,11,2,1,97500,False,False,False


# Splitting the data into X and y

In [17]:
x = df.drop('price', axis=1)
y = df['price']

In [18]:
np.random.seed(42)

# Split data into train and test sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [19]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((150826, 15), (37707, 15), (150826,), (37707,))

In [20]:
# Set the seed
np.random.seed(42)

# Initialize the Model
model = RandomForestRegressor(n_jobs=-1, random_state=42,max_samples=20000)

# Fit the model
model.fit(x_train, y_train)

# Model score
model.score(x_test, y_test)

0.1443094604271794

In [21]:
model.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'squared_error',
 'max_depth': None,
 'max_features': 1.0,
 'max_leaf_nodes': None,
 'max_samples': 20000,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'monotonic_cst': None,
 'n_estimators': 100,
 'n_jobs': -1,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}

# Evaluation Function

In [22]:
def rmse(y_test, y_preds):
    """
    Calculate root mean squared error between predictions and true labels
    """
    return np.sqrt(mean_squared_error(y_test, y_preds))

# Create an evaluation function
def evaluate_model(model):
    """
    Performs evaluation comparison on y_true labels vs. y_pred labels on a regresion model.
    """
    y_preds = model.predict(x_test)
    scores = {'Root Mean Squared Error (RMSE)': rmse(y_test, y_preds)}

    return scores

In [23]:
evaluate_model(model)

{'Root Mean Squared Error (RMSE)': 68982.68863867597}

# Hyperparameter tuning with RandomizedSearchCV

In [24]:
from sklearn.model_selection import RandomizedSearchCV

# Define the parameter grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_features': ['sqrt', 'log2'],
    'max_depth': [10, 20, 30],
    'min_samples_split': [5, 10],
    'min_samples_leaf': [2, 4],
    'bootstrap': [True]
}

# Initialize RandomizedSearchCV
rs_model = RandomizedSearchCV(estimator=model,
                                   param_distributions=param_grid,
                                   n_iter=50,  # Number of parameter settings that are sampled
                                   cv=5,  # 5-fold cross-validation
                                   random_state=42)

# Fit RandomizedSearchCV
rs_model.fit(x_train, y_train)

# Best parameters found and best score
print(f'Best Parameters are: {rs_model.best_params_}')
print(f'Best Score is: {rs_model.best_score_}')

/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Best Parameters are: {'n_estimators': 300, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_features': 'log2', 'max_depth': 30, 'bootstrap': True}
Best Score is: 0.14302622346096722


In [25]:
evaluate_model(rs_model)

{'Root Mean Squared Error (RMSE)': 68101.98970112349}

# Test on test data

In [26]:
# Read the test data
test_data = pd.read_csv('/content/drive/MyDrive/Used Car Price Prediction/data/test.csv')
test_data.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title
0,188533,Land,Rover LR2 Base,2015,98000,Gasoline,240.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,6-Speed A/T,White,Beige,None reported,Yes
1,188534,Land,Rover Defender SE,2020,9142,Hybrid,395.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,8-Speed A/T,Silver,Black,None reported,Yes
2,188535,Ford,Expedition Limited,2022,28121,Gasoline,3.5L V6 24V PDI DOHC Twin Turbo,10-Speed Automatic,White,Ebony,None reported,NaN
3,188536,Audi,A6 2.0T Sport,2016,61258,Gasoline,2.0 Liter TFSI,Automatic,Silician Yellow,Black,None reported,NaN
4,188537,Audi,A6 2.0T Premium Plus,2018,59000,Gasoline,252.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,A/T,Gray,Black,None reported,Yes


# Preprocess Test Data

In [27]:
test_data.isna().sum()

,0
id,0
brand,0
model,0
model_year,0
milage,0
fuel_type,3383
engine,0
transmission,0
ext_col,0
int_col,0


In [28]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125690 entries, 0 to 125689
Data columns (total 12 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            125690 non-null  int64 
 1   brand         125690 non-null  object
 2   model         125690 non-null  object
 3   model_year    125690 non-null  int64 
 4   milage        125690 non-null  int64 
 5   fuel_type     122307 non-null  object
 6   engine        125690 non-null  object
 7   transmission  125690 non-null  object
 8   ext_col       125690 non-null  object
 9   int_col       125690 non-null  object
 10  accident      124058 non-null  object
 11  clean_title   111451 non-null  object
dtypes: int64(3), object(9)
memory usage: 11.5+ MB


In [29]:
# Find the columns which contains strings
for label, content in test_data.items():
    if pd.api.types.is_string_dtype(content):
        print(label)

brand
model
engine
transmission
ext_col
int_col


In [30]:
# This will turn all the string value into category values
for label, content in test_data.items():
    if pd.api.types.is_string_dtype(content):
        test_data[label] = content.astype('category').cat.as_ordered()

In [31]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125690 entries, 0 to 125689
Data columns (total 12 columns):
 #   Column        Non-Null Count   Dtype   
---  ------        --------------   -----   
 0   id            125690 non-null  int64   
 1   brand         125690 non-null  category
 2   model         125690 non-null  category
 3   model_year    125690 non-null  int64   
 4   milage        125690 non-null  int64   
 5   fuel_type     122307 non-null  object  
 6   engine        125690 non-null  category
 7   transmission  125690 non-null  category
 8   ext_col       125690 non-null  category
 9   int_col       125690 non-null  category
 10  accident      124058 non-null  object  
 11  clean_title   111451 non-null  object  
dtypes: category(6), int64(3), object(3)
memory usage: 7.1+ MB


In [32]:
# Convert object columns to categorical
test_data['fuel_type'] = test_data['fuel_type'].astype('category')
test_data['accident'] = test_data['accident'].astype('category')
test_data['clean_title'] = test_data['clean_title'].astype('category')

In [33]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125690 entries, 0 to 125689
Data columns (total 12 columns):
 #   Column        Non-Null Count   Dtype   
---  ------        --------------   -----   
 0   id            125690 non-null  int64   
 1   brand         125690 non-null  category
 2   model         125690 non-null  category
 3   model_year    125690 non-null  int64   
 4   milage        125690 non-null  int64   
 5   fuel_type     122307 non-null  category
 6   engine        125690 non-null  category
 7   transmission  125690 non-null  category
 8   ext_col       125690 non-null  category
 9   int_col       125690 non-null  category
 10  accident      124058 non-null  category
 11  clean_title   111451 non-null  category
dtypes: category(9), int64(3)
memory usage: 4.6 MB


In [34]:
test_data.isna().sum()

,0
id,0
brand,0
model,0
model_year,0
milage,0
fuel_type,3383
engine,0
transmission,0
ext_col,0
int_col,0


In [35]:
# Turn categorical variables into numbers and fill missing
for label, content in test_data.items():
  if not pd.api.types.is_numeric_dtype(content):
    # Turn categories into numbers and add +1 (only for non-null entries)
    test_data[label] = pd.Categorical(content).codes + 1
    # Check if there are any missing values in the column
    if content.isna().sum() > 0:
      # Add binary column to indicate whether sample had missing values
      test_data[label + '_is_missing'] = pd.isnull(content)

In [36]:
test_data.isna().sum()

,0
id,0
brand,0
model,0
model_year,0
milage,0
fuel_type,0
engine,0
transmission,0
ext_col,0
int_col,0


In [37]:
# We can find how the columns differ using sets
set(x_train.columns) - set(test_data.columns)

set()

In [38]:
# Make predictions on updated test data
test_preds = rs_model.predict(test_data)

In [39]:
df_preds = pd.DataFrame()
df_preds["id"] = test_data["id"]
df_preds["price"] = test_preds
df_preds

,id,price
0,188533,21955.578637
1,188534,120165.571817
2,188535,72897.489914
3,188536,33955.826009
4,188537,33674.245838
...,...,...
125685,314218,31966.784986
125686,314219,68285.580286
125687,314220,29947.175720
125688,314221,23680.943592


In [40]:
# Export prediction data
df_preds.to_csv("/content/drive/MyDrive/Used Car Price Prediction/data/test_predictions.csv", index=False)